In [1]:
!pip install scikit-learn==0.21.3

In [2]:
# Import the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.metrics import mean_squared_error

warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('max_column', 1000)

/home/philmassie/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:53: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [4]:
train = pd.read_csv('Train_maskedv2.csv')
test = pd.read_csv('Test_maskedv2.csv')
sample = pd.read_csv('samplesubmissionv2.csv')
description = pd.read_csv('variable_descriptions_v2.csv')

In [5]:
pd.set_option('max_colwidth', 200)
pd.set_option('max_info_rows', 1000)

In [6]:
train.drop(columns=['dw_12', 'dw_13', 'lan_13'], inplace=True)
test.drop(columns=['dw_12', 'dw_13', 'lan_13'], inplace=True)

In [7]:
train_len = len(train)
data=pd.concat([train,test])

In [8]:
data['rich'] = data['car_01']+data['stv_01']+data['psa_01']+data['dw_02']+data['lln_00']
data['poor'] = data['car_00'] +data['stv_00']+data['psa_00']+data['dw_01']+data['lln_01']

In [9]:
data['household_size'] =data['total_individuals'] / data['total_households']

In [10]:
from sklearn.cluster import KMeans
columns=data.drop(["target_pct_vunerable","ward"],1).columns

data_km=data[columns].copy()

data_km["total_households"]/=data_km["total_households"].max()
data_km["total_individuals"]/=data_km["total_individuals"].max()

km=KMeans(15,random_state=2019)
data["cluster"]=km.fit_predict(data_km[columns])

In [11]:
data['cluster'].value_counts()

7     493
3     468
0     409
8     392
13    381
12    376
9     359
4     343
2     337
14    298
11    213
6     187
10     17
5       2
1       1
Name: cluster, dtype: int64

In [12]:
train = data[:train_len]
test = data[train_len:]

In [13]:
_id = test['ward']
test.drop(columns=['target_pct_vunerable','ward'], inplace=True)
train.drop(columns=['ward'], inplace=True)

In [14]:
train.shape, test.shape

((3174, 50), (1102, 49))

In [15]:
train['total_households'] = np.log10(train['total_households'])
test['total_households'] = np.log10(test['total_households'])

train['total_individuals'] = np.log10(train['total_individuals'])
test['total_individuals'] = np.log10(test['total_individuals'])

In [16]:
X = train.drop(columns=['target_pct_vunerable'])
y = train['target_pct_vunerable']

In [17]:
X.columns

Index(['car_00', 'car_01', 'dw_00', 'dw_01', 'dw_02', 'dw_03', 'dw_04',
       'dw_05', 'dw_06', 'dw_07', 'dw_08', 'dw_09', 'dw_10', 'dw_11', 'lan_00',
       'lan_01', 'lan_02', 'lan_03', 'lan_04', 'lan_05', 'lan_06', 'lan_07',
       'lan_08', 'lan_09', 'lan_10', 'lan_11', 'lan_12', 'lan_14', 'lgt_00',
       'lln_00', 'lln_01', 'pg_00', 'pg_01', 'pg_02', 'pg_03', 'pg_04',
       'psa_00', 'psa_01', 'psa_02', 'psa_03', 'psa_04', 'stv_00', 'stv_01',
       'total_households', 'total_individuals', 'rich', 'poor',
       'household_size', 'cluster'],
      dtype='object')

In [18]:
X.isna().sum().any()

False

In [19]:
col = ['car_00', 'car_01', 'dw_00', 'dw_01', 'dw_02', 'dw_03', 'dw_04',
       'lan_08', 'lan_09', 'lan_10', 'lan_11', 'lan_12', 'lan_14', 'lgt_00',
       'dw_05', 'dw_06', 'dw_07', 'dw_08', 'dw_09', 'dw_10', 'dw_11', 'lan_00',
       'lan_01', 'lan_02', 'lan_03', 'lan_04', 'lan_05', 'lan_06', 'lan_07',
       'lln_00', 'lln_01', 'pg_00', 'pg_01', 'pg_02', 'pg_03', 'pg_04',
       'psa_00', 'psa_01', 'psa_02', 'psa_03', 'psa_04', 'stv_00', 'stv_01',
        'rich', 'poor']

In [20]:
X[col] = X[col].round(2)
test[col] = test[col].round(2)

In [21]:
categorical_features_indices = np.where(X.dtypes != np.float)[0]; categorical_features_indices

array([48])

In [22]:
X['cluster'].value_counts()

7     413
13    381
12    369
9     359
3     346
4     343
2     337
14    297
6     175
8     124
0      13
11     11
10      5
1       1
Name: cluster, dtype: int64

In [23]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit

testsplit_store=[]
test_store=[]
fold=KFold(n_splits=15, shuffle=True, random_state=123456)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    cat = CatBoostRegressor(n_estimators=10000,eval_metric='RMSE', learning_rate=0.0801032, random_seed= 123456, l2_leaf_reg=4, use_best_model=True)
    cat.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=300,verbose=100, cat_features=categorical_features_indices)
    predict = cat.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,predict)))
    testsplit_store.append(np.sqrt(mean_squared_error(y_test,predict)))
    pred = cat.predict(test)
    test_store.append(pred)

0:	learn: 12.1780208	test: 12.1780208	test1: 13.1044743	best: 13.1044743 (0)	total: 55.1ms	remaining: 9m 10s
100:	learn: 5.6174210	test: 5.6132629	test1: 5.5393814	best: 5.5390735 (95)	total: 531ms	remaining: 52s
200:	learn: 4.4863793	test: 4.4817942	test1: 5.2740907	best: 5.2546821 (190)	total: 1s	remaining: 48.9s
300:	learn: 3.7980081	test: 3.8010415	test1: 5.1792462	best: 5.1792462 (300)	total: 1.48s	remaining: 47.6s
400:	learn: 3.3354550	test: 3.3414800	test1: 5.1836335	best: 5.1401847 (337)	total: 1.94s	remaining: 46.4s
500:	learn: 2.9796511	test: 2.9941472	test1: 5.1514855	best: 5.1401847 (337)	total: 2.41s	remaining: 45.8s
600:	learn: 2.6850130	test: 2.7018982	test1: 5.1083993	best: 5.0993322 (565)	total: 2.87s	remaining: 45s
700:	learn: 2.4217856	test: 2.4425302	test1: 5.0739039	best: 5.0729149 (697)	total: 3.35s	remaining: 44.5s
800:	learn: 2.2032932	test: 2.2265055	test1: 5.0774592	best: 5.0656514 (706)	total: 3.83s	remaining: 44.1s
900:	learn: 2.0289666	test: 2.0531066	test1

700:	learn: 2.4133965	test: 2.4406559	test1: 5.0535571	best: 5.0535571 (700)	total: 4.38s	remaining: 58.1s
800:	learn: 2.1704105	test: 2.2034776	test1: 5.0672229	best: 5.0491884 (735)	total: 4.93s	remaining: 56.7s
900:	learn: 1.9928762	test: 2.0302171	test1: 5.0508519	best: 5.0491884 (735)	total: 5.53s	remaining: 55.8s
1000:	learn: 1.8254726	test: 1.8660332	test1: 5.0380197	best: 5.0343727 (975)	total: 6.14s	remaining: 55.2s
1100:	learn: 1.6757789	test: 1.7174212	test1: 5.0426092	best: 5.0325298 (1006)	total: 6.68s	remaining: 54s
1200:	learn: 1.5342039	test: 1.5788098	test1: 5.0383624	best: 5.0311085 (1134)	total: 7.23s	remaining: 53s
1300:	learn: 1.3941805	test: 1.4419058	test1: 5.0515694	best: 5.0311085 (1134)	total: 7.8s	remaining: 52.1s
1400:	learn: 1.2836839	test: 1.3330413	test1: 5.0458559	best: 5.0311085 (1134)	total: 8.5s	remaining: 52.2s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 5.031108518
bestIteration = 1134

Shrink model to first 1135 iterations.
e

1400:	learn: 1.3266726	test: 1.3627477	test1: 5.3168463	best: 5.3148789 (1397)	total: 7.46s	remaining: 45.8s
1500:	learn: 1.2363255	test: 1.2748659	test1: 5.3026239	best: 5.3025786 (1488)	total: 7.99s	remaining: 45.2s
1600:	learn: 1.1431307	test: 1.1859325	test1: 5.3016591	best: 5.3002502 (1565)	total: 8.52s	remaining: 44.7s
1700:	learn: 1.0580258	test: 1.1049395	test1: 5.2872777	best: 5.2872444 (1698)	total: 9.05s	remaining: 44.2s
1800:	learn: 0.9888531	test: 1.0386334	test1: 5.2911644	best: 5.2858618 (1732)	total: 9.6s	remaining: 43.7s
1900:	learn: 0.9239806	test: 0.9754426	test1: 5.2812300	best: 5.2781391 (1880)	total: 10.1s	remaining: 43.2s
2000:	learn: 0.8705115	test: 0.9244677	test1: 5.2823937	best: 5.2775297 (1984)	total: 10.7s	remaining: 42.8s
2100:	learn: 0.8120697	test: 0.8694487	test1: 5.2787084	best: 5.2774778 (2081)	total: 11.2s	remaining: 42.3s
2200:	learn: 0.7572808	test: 0.8182009	test1: 5.2763623	best: 5.2738797 (2187)	total: 11.8s	remaining: 41.7s
2300:	learn: 0.70875

In [24]:
np.mean(testsplit_store)

5.645641368285069

In [25]:
submit_prep = {"ward": _id, 'target_pct_vunerable': np.mean(test_store, 0)}
submission = pd.DataFrame(data = submit_prep)

In [26]:
submission.head()

,ward,target_pct_vunerable
0,00A93O0,18.644497
1,01V40KO,30.680925
2,01ZHVW0,6.572926
3,028YQJX,1.731807
4,02G8W11,0.492371


In [27]:
submission.to_csv('covid_19_best_Amy.csv', index=False)

In [28]:
## Check for the feature importance 
#fea_imp = pd.DataFrame({'imp':cat.feature_importances_, 'col': X.columns})
#fea_imp = fea_imp.sort_values(['imp', 'col'], ascending=[True, False]).iloc[-60:]
#_ = fea_imp.plot(kind='barh', x='col', y='imp', figsize=(20, 10))
#plt.savefig('catboost_feature_importance.png')